In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from austin_animal_center import AustinAnimalCenter
import base64


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "3"
shelter = AustinAnimalCenter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})).drop('_id', axis = 1)
df = df.sort_values(by=['1'])
rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Title and Image
    html.Center(children=[
        html.B(html.H1('SNHU CS-340 Dashboard')),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
        # Unique identifier
        html.Center(html.H4('Dustin Jackson 2022')),
    ]),
    html.Hr(),
    
    # Radio buttons for each type of rescue animal in the shelter
    dcc.RadioItems(
                id='rescue-type-filter',
                options=[{'label': i, 'value': i} for i in rescue_types],
                value='Reset',
                labelStyle={'display': 'inline-block'}
            ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        # Features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
    # First row with Pie chart and Map
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    
    # Second row with Histogram
    html.Div(
        id='second-graph-id',
        className='col s12 m6',
        ),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback for the radio buttons to filter data based on rescue type
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-filter', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                & (df.sex_upon_outcome == 'Intact Female')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300))]
        
    return dff.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# First graph callback: Pie chart with the percentage of animal breed in the viewport data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    df_grouped = dff[['breed', '1']].groupby(['breed']).agg('count').reset_index()
    df_grouped = df_grouped.rename(columns = {'1': 'Count'})
    
    fig = px.pie(df_grouped, values='Count', names='breed', title="Animal Types")
    return [
        dcc.Graph(figure=fig)
    ]

# Map callback that displays markers for the locations of animals from them viewport data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")] +
            # Marker with tool tip and popup
            [dl.Marker(position=[row['location_lat'],row['location_long']], children=[
                dl.Tooltip(row['breed']),
                dl.Popup([
                     html.H1("Animal Name"),
                     html.P(row['name'])
                ])
            ]) for index, row in dff.iterrows()]
        )
    ]

# Second Graph callback: Histogram displays ages of animals from the viewport data
@app.callback(
    Output('second-graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs2(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.histogram(dff, x="age_upon_outcome_in_weeks")
    return [
        dcc.Graph(figure=fig)
    ]

app


ModuleNotFoundError: No module named 'austin_animal_center'